<a href="https://colab.research.google.com/github/hellocybernetics/TensorFlow_Eager_Execution_Tutorials/blob/master/tutorials/02_intermediate/Convolutinal_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf-nightly-2.0-preview

    100% |████████████████████████████████| 74.3MB 579kB/s 
    100% |████████████████████████████████| 3.2MB 9.8MB/s 
    100% |████████████████████████████████| 245kB 17.6MB/s 
    100% |████████████████████████████████| 61kB 26.1MB/s 
    100% |████████████████████████████████| 61kB 25.4MB/s 
    100% |████████████████████████████████| 112kB 18.8MB/s 


In [0]:
import tensorflow as tf
import numpy as np

L = tf.keras.layers

In [3]:
# Hyper parameters
num_epochs = 10
num_classes = 10
batch_size = 1024
learning_rate = 0.001

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

print("training_data\n", x_train.shape)
print("test_data\n", x_test.shape)
print("training_label\n", y_train.shape)
print("test_label\n", y_test.shape)

170500096/170498071 [==============================] - 32s 0us/step
training_data
 (50000, 32, 32, 3)
test_data
 (10000, 32, 32, 3)
training_label
 (50000, 1)
test_label
 (10000, 1)


In [26]:
train_dataset = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .batch(batch_size)
    .shuffle(10000)
)

train_dataset = (
    train_dataset.map(lambda x, y: 
                      (tf.div_no_nan(tf.cast(x, tf.float32), 255.0), 
                       tf.reshape(tf.one_hot(y, 10), (-1, 10))))
)

print(train_dataset)

<MapDataset shapes: ((None, 32, 32, 3), (None, 10)), types: (tf.float32, tf.float32)>


In [27]:
test_dataset = (
    tf.data.Dataset.from_tensor_slices((x_test, y_test))
    .batch(1000)
    .shuffle(10000)
)
test_dataset = (
    test_dataset.map(lambda x, y: 
                      (tf.div_no_nan(tf.cast(x, tf.float32), 255.0), 
                       tf.reshape(tf.one_hot(y, 10), (-1, 10))))
)
print(test_dataset)

<MapDataset shapes: ((None, 32, 32, 3), (None, 10)), types: (tf.float32, tf.float32)>


In [0]:
class Cifar10Model(tf.keras.Model):
    def __init__(self):
        super(Cifar10Model, self).__init__(name='cifar_cnn')
        
        self.conv_block1 = tf.keras.Sequential([
            L.Conv2D(
                8, 
                5,
                padding='same',
                activation=tf.nn.relu,
                kernel_regularizer=tf.keras.regularizers.l2(l=0.001),
            ),
            L.MaxPooling2D(
                (3, 3), 
                (2, 2), 
                padding='same'
            ),
        ])

        self.conv_block2 = tf.keras.Sequential([
            L.Conv2D(
                16, 
                5,
                padding='same',
                activation=tf.nn.relu,
                kernel_regularizer=tf.keras.regularizers.l2(l=0.001),
            ),
            L.MaxPooling2D(
                (3, 3), 
                (2, 2), 
                padding='same',
            ),
        ])
        
        self.conv_block3 = tf.keras.Sequential([
            L.Conv2D(
                32, 
                5,
                padding='same',
                activation=tf.nn.relu,
                kernel_regularizer=tf.keras.regularizers.l2(l=0.001),
            ),
            L.MaxPooling2D(
                (3, 3), 
                (2, 2), 
                padding='same',
            ),
        ])
        
        self.flatten = L.Flatten()
        self.fc1 = L.Dense(
            256, 
            activation=tf.nn.relu,
            kernel_regularizer=tf.keras.regularizers.l2(l=0.001))
        self.dropout = L.Dropout(0.8)
        self.fc2 = L.Dense(10)
        self.softmax = L.Softmax()

    def call(self, x, training=False):
        x = self.conv_block1(x, training=training)
        x = self.conv_block2(x, training=training)
        x = self.conv_block3(x, training=training)
        x = self.flatten(x)
        x = self.dropout(self.fc1(x), training=training)
        x = self.fc2(x)
        return self.softmax(x)

In [0]:
model = Cifar10Model()

def loss_fn(y, y_pre):
    return tf.keras.losses.categorical_crossentropy(y, y_pre)

def accuracy(y, y_pre):
    return tf.keras.metrics.categorical_accuracy(y, y_pre)

optimizer = tf.keras.optimizers.Adam(learning_rate)

In [41]:
for j in range(num_epochs):
    
    running_loss = 0
    running_acc = 0

    for i, (x_, y_) in enumerate(train_dataset):
        
        with tf.GradientTape() as tape:
            y_pre = model(x_, training=True)
            loss = loss_fn(y_, y_pre)
        acc = accuracy(y_, y_pre)
        grads = tape.gradient(loss, model.variables)
        optimizer.apply_gradients(zip(grads, model.variables))
        running_loss += tf.reduce_mean(loss)
        running_acc += tf.reduce_mean(acc)
    
    print("-----epoch {} -----".format(j + 1))
    print("loss: ", running_loss.numpy()/(i + 1))
    print("acc: ", running_acc.numpy()/(i + 1))    

-----epoch 1 -----
loss:  2.159227176588409
acc:  0.18321771037821866
-----epoch 2 -----
loss:  1.8324200377172353
acc:  0.3226679782478177
-----epoch 3 -----
loss:  1.6927250453404017
acc:  0.3773806824976084
-----epoch 4 -----
loss:  1.5976887917032048
acc:  0.41615400508958467
-----epoch 5 -----
loss:  1.536187775281011
acc:  0.4423094768913425
-----epoch 6 -----
loss:  1.48232144725566
acc:  0.461150344537229
-----epoch 7 -----
loss:  1.4467300103635203
acc:  0.4776394513188576
-----epoch 8 -----
loss:  1.414776393345424
acc:  0.4917148278684032
-----epoch 9 -----
loss:  1.3857308212591677
acc:  0.5027416774204799
-----epoch 10 -----
loss:  1.3624782951510683
acc:  0.5130796237867705


In [42]:
test_accuracy = 0
for i, (x_, y_) in enumerate(test_dataset):
    y_pre = model(x_)
    test_accuracy += tf.reduce_mean(accuracy(y_, y_pre))
test_accuracy /= i + 1

print("test accuracy {:0.3f}".format(test_accuracy.numpy()))

test accuracy 0.556
